In [1]:
from IPython.display import display
from typing import TypedDict, Unpack
from pprint import pprint

There are plenty of tricks in Python to remove all kinds of boiler plate and improve the DX (developer experience).
Here we'll go over some of them

### Display the narrative via `_repr_html_`

In [2]:
from fhirpy import SyncFHIRClient
from fhirpy.base.exceptions import OperationOutcome
import r4
FHIR_SERVER_API = "http://hapi.fhir.org/baseR4"

client = SyncFHIRClient(FHIR_SERVER_API)
try:
    response = client.resources("Patient").search(birthdate="1970-01-01", name="Doe").limit(1).first()
    patient = r4.Patient.parse_obj(response)
except OperationOutcome as e:
    display(r4.OperationOutcome(**e.resource))
else:
    display(patient) # this presents the narrative as HTML in the notebook

Date of birth,01 January 1970


### Build Parameters from a dictionary

In [4]:
# Now you can define parameters like this:    
class MyKwargs(TypedDict):
    name: r4.string
    birthdate: r4.date

# Define this is once
class SmartParameters(r4.Parameters):

    @classmethod
    def from_kwargs(cls, **kwargs:Unpack[MyKwargs]):
        """Create a Parameters resource from a dictionary with keys and fhir typed values."""
        return cls.parse_obj({
                "resourceType": "Parameters",
                "parameter": [
                    {"name": k, f"value{MyKwargs.__annotations__[k].__name__.capitalize()}": v} for k,v in kwargs.items()
                ]
            })
my_params = SmartParameters.from_kwargs(name=r4.string("John"), birthdate=r4.date("1970-01-01"))

pprint(my_params)

SmartParameters(resourceType='Parameters', parameter=[ParametersParameter(name='name', valueString='John'), ParametersParameter(name='birthdate', valueDate=Date(year=1970, month=1, day=1))])


### `exclude_none` by default when serializing to JSON

In [5]:
print(my_params.json(indent=2))

{
  "resourceType": "Parameters",
  "parameter": [
    {
      "name": "name",
      "part": [],
      "valueString": "John",
      "extension": [],
      "modifierExtension": []
    },
    {
      "name": "birthdate",
      "part": [],
      "valueDate": "1970-01-01",
      "extension": [],
      "modifierExtension": []
    }
  ]
}


### Build a custom date type that supports `YYYY`, `YYYY-MM` and `YYYY-MM-DD`

In [7]:
from fhirpy import SyncFHIRClient
from fhirpy.base.exceptions import OperationOutcome
FHIR_SERVER_API = "http://hapi.fhir.org/baseR4"

client = SyncFHIRClient(FHIR_SERVER_API)
try:
    response = client.resources("Patient").search(birthdate="1970-01-01", name="Doe").limit(1).first()
    patient = r4.Patient.parse_obj(response)
    assert patient.birthDate is not None
    display(patient.birthDate)
    display(patient.birthDate.year)
except OperationOutcome as e:
    display(r4.OperationOutcome(**e.resource))

Date(year=1970, month=1, day=1)

1970

In [9]:
patient_with_birthyearmonth = r4.Patient.parse_obj({**patient.dict(), "birthDate": "1970-01"})
assert patient_with_birthyearmonth.birthDate is not None
print(patient_with_birthyearmonth.birthDate)
print(f"year={patient_with_birthyearmonth.birthDate.year}", f"month={patient_with_birthyearmonth.birthDate.month}")

1970-1
year=1970 month=1


Other things you might do:
- display only the 'summary' fields in `__repr__`
- add lookup mechanisms for extensions
- automatically determine the type for polymorphic fields (ex. `Observation.value[x]`)
- add properties to access slices